In [11]:
file = 'data/directed/ego-twitter/train.in'
N = 10

In [12]:
import tools

In [13]:
train_set, nodes, poss_set, neg_set = tools.sample_structural(file, N, directed=True)

In [14]:
from graph_tool.all import *
g = Graph(directed=False)
g.add_vertex(2*max(nodes) + 1)

<generator object <genexpr> at 0x7fa0c7b42b40>

In [15]:
print 2*max(nodes) + 1

46741


In [16]:
groups = g.new_vertex_property("int")

for u in g.vertices():
    groups[u] = int(u) % 2

In [17]:
s = 0
for edge in train_set:
    if edge not in poss_set:
        u, w = map(int, edge.split())
        if (str(w) + ' ' + str(u) in train_set) and (str(w) + ' ' + str(u) not in poss_set):
            g.add_edge(g.vertex(2*u - 1), g.vertex(2*w - 1))
            g.add_edge(g.vertex(2*u), g.vertex(2*w))
            s += 1
        g.add_edge(g.vertex(2*u - 1), g.vertex(2*w))
print s

436


In [18]:
pos = sfdp_layout(g, max_iter=4)

In [19]:
from sklearn.metrics import roc_auc_score

In [20]:
features = tools.TopologicalFeatures(g, pos, directed=True, gap=0)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.dist])
roc_auc_score(Y, X)

0.7180809777201741

In [ ]:
from graph_tool.all import *
g = Graph(directed=False)
g.add_vertex(max(nodes) + 1)

for edge in train_set:
    if edge not in poss_set:
        u, w = map(int, edge.split())
        g.add_edge(g.vertex(u), g.vertex(w))


features = tools.TopologicalFeatures(g)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.preferential_attachment])
roc_auc_score(Y, X)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random', random_state=0)

matrix = tools.make_sparse_matrix(train_set, nodes, poss_set, directed=True)

features = tools.MFFeatures(model, matrix)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.score])
roc_auc_score(Y, X)

In [ ]:
from scipy.sparse import linalg
import numpy

matrix = tools.make_sparse_matrix(train_set, nodes, poss_set, directed=True)
U, s, Vh = linalg.svds(matrix.asfptype(), k=30)

def score(u, w):
    return numpy.dot(U[u] * s, Vh.T[w])

X, Y = tools.make_dataset(poss_set, neg_set, 
                    [score])
roc_auc_score(Y, X)

# Auto cross-validation

In [1]:
import cross_validation

file = 'data/directed/ego-twitter/train.in'
N = 10
cross_validation.cross_validation(file, N, 1, directed=True, max_iter=4, gap=0, verbose=True)

Positioning level: 0 46741 with K = 112.125313443 ...
avg edge distance: 230.183027127
gap 0
sfdp-bipartite-repulse-allrepulse-all : nan (+/- nan)
PA : 0.50 (+/- 0.00)
sfdp-bipartite-repulse-fellowsrepulse-fellows : nan (+/- nan)
NMF-10 : 0.70 (+/- 0.00)
sfdp-bipartite-repulse-fellowsrepulse-all : nan (+/- nan)
sfdp-bipartite-repulse-aliensrepulse-aliens : nan (+/- nan)
sfdp-directed : 0.79 (+/- 0.00)
svds-30 : 0.69 (+/- 0.00)
sfdp-bipartite-repulse-allrepulse-fellows : nan (+/- nan)
sfdp-bipartite-repulse-aliensrepulse-all : nan (+/- nan)
sfdp-bipartite-repulse-aliensrepulse-fellows : nan (+/- nan)
sfdp-bipartite-repulse-allrepulse-aliens : nan (+/- nan)
sfdp-bipartite-repulse-fellowsrepulse-aliens : nan (+/- nan)
sfdp-default : 0.50 (+/- 0.00)


/usr/lib/python2.7/dist-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/lib/python2.7/dist-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/lib/python2.7/dist-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/usr/lib/python2.7/dist-packages/numpy/core/_methods.py:94: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/lib/python2.7/dist-packages/numpy/core/_methods.py:116: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
